In [ ]:
from typing import List, Dict, Optional
import re

class AddressNormalizer:
    def __init__(self):
        # 一般的な表記揺れパターン
        self.patterns = {
            r'１|一': '1',
            r'２|二': '2',
            r'３|三': '3',
            r'４|四': '4',
            r'５|五': '5',
            r'６|六': '6',
            r'７|七': '7',
            r'８|八': '8',
            r'９|九': '9',
            r'０': '0',
            r'[都道府県]?(東京都)': r'\1',
            r'([都道府県])$': r'\1',
            r'([市区町村])$': r'\1',
            r'([0-9]+)ー([0-9]+)': r'\1-\2',  # 横棒の統一
            r'([0-9]+)番地': r'\1',
            r'([0-9]+)号': r'\1',
            r'([0-9]+)丁目': r'\1',
            r'([0-9]+)番': r'\1',
            r'([0-9]+)\s*の\s*([0-9]+)': r'\1-\2',  # 「の」を横棒に変換
            r'([0-9]+)百': lambda m: str(int(m.group(1)) * 100),  # 百の処理
            r'([0-9]+)十([0-9])?': lambda m: str(int(m.group(1)) * 10 + (int(m.group(2)) if m.group(2) else 0)),  # 十の処理
        }

        # 都道府県の正式名称マッピング
        self.prefecture_mapping = {
            '北海道': '北海道',
            '青森': '青森県',
            '岩手': '岩手県',
            '宮城': '宮城県',
            '秋田': '秋田県',
            '山形': '山形県',
            '福島': '福島県',
            '茨城': '茨城県',
            '栃木': '栃木県',
            '群馬': '群馬県',
            '埼玉': '埼玉県',
            '千葉': '千葉県',
            '東京': '東京都',
            '神奈川': '神奈川県',
            '新潟': '新潟県',
            '富山': '富山県',
            '石川': '石川県',
            '福井': '福井県',
            '山梨': '山梨県',
            '長野': '長野県',
            '岐阜': '岐阜県',
            '静岡': '静岡県',
            '愛知': '愛知県',
            '三重': '三重県',
            '滋賀': '滋賀県',
            '京都': '京都府',
            '大阪': '大阪府',
            '兵庫': '兵庫県',
            '奈良': '奈良県',
            '和歌山': '和歌山県',
            '鳥取': '鳥取県',
            '島根': '島根県',
            '岡山': '岡山県',
            '広島': '広島県',
            '山口': '山口県',
            '徳島': '徳島県',
            '香川': '香川県',
            '愛媛': '愛媛県',
            '高知': '高知県',
            '福岡': '福岡県',
            '佐賀': '佐賀県',
            '長崎': '長崎県',
            '熊本': '熊本県',
            '大分': '大分県',
            '宮崎': '宮崎県',
            '鹿児島': '鹿児島県',
            '沖縄': '沖縄県'
        }


    def normalize(self, address: str) -> str:
        """
        住所文字列を正規化する

        Args:
            address (str): 正規化したい住所文字列

        Returns:
            str: 正規化された住所文字列
        """
        # 全角スペースを半角に統一
        normalized = address.replace('　', ' ').strip()

        # 建物名を削除（必要に応じてコメントアウトを解除）
        # normalized = re.split(r'\s+(?=[^\s]*$)', normalized)[0]

        # 漢数字と全角数字を半角数字に変換
        for pattern, replacement in self.patterns.items():
            if callable(replacement):
                # 関数による置換（百、十などの処理）
                normalized = re.sub(pattern, replacement, normalized)
            else:
                normalized = re.sub(pattern, replacement, normalized)

        # 都道府県名の正規化
        for pref, formal_name in self.prefecture_mapping.items():
            if normalized.startswith(pref):
                normalized = normalized.replace(pref, formal_name, 1)
                break

        return normalized

    def compare_addresses(self, addr1: str, addr2: str) -> float:
        """
        2つの住所の類似度を計算する

        Args:
            addr1 (str): 比較する住所1
            addr2 (str): 比較する住所2

        Returns:
            float: 類似度スコア (0.0 ~ 1.0)
        """
        # 両方の住所を正規化
        norm1 = self.normalize(addr1)
        norm2 = self.normalize(addr2)

        # 文字列の編集距離を計算
        distance = self._levenshtein_distance(norm1, norm2)
        max_length = max(len(norm1), len(norm2))

        # 類似度を計算 (1に近いほど似ている)
        similarity = 1 - (distance / max_length)
        return similarity

    def _levenshtein_distance(self, s1: str, s2: str) -> int:
        """
        レーベンシュタイン距離を計算する
        """
        if len(s1) < len(s2):
            return self._levenshtein_distance(s2, s1)

        if len(s2) == 0:
            return len(s1)

        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row

        return previous_row[-1]

# 使用例
if __name__ == "__main__":
    normalizer = AddressNormalizer()

    # 表記揺れの例
    test_cases = [
        # 東京都の例
        {
            "inputs": [
                "東京都千代田区丸の内１ー１",
                "東京 千代田区丸の内1-1",
                "東京都千代田区丸の内一番地",
                "東京都千代田区丸の内1丁目1番",
                "東京都千代田区丸の内1-1-1",
            ],
            "expected": "東京都千代田区丸の内1-1"
        },
        # 神奈川県の例
        {
            "inputs": [
                "神奈川県横浜市西区みなとみらい２ー３ー５",
                "神奈川 横浜市西区みなとみらい2-3-5",
                "神奈川県横浜市西区みなとみらい二丁目三番五号",
                "横浜市西区みなとみらい2丁目3番5号",
                "神奈川県横浜市西区みなとみらい2-3-5"
            ],
            "expected": "神奈川県横浜市西区みなとみらい2-3-5"
        },
        # 大阪府の例
        {
            "inputs": [
                "大阪府大阪市中央区心斎橋筋１ー１ー１０",
                "大阪 大阪市中央区心斎橋筋1-1-10",
                "大阪府大阪市中央区心斎橋筋一丁目一番十号",
                "大阪市中央区心斎橋筋1丁目1番10号",
            ],
            "expected": "大阪府大阪市中央区心斎橋筋1-1-10"
        },
        # 住居表示と地番表示の混在例
        {
            "inputs": [
                "神奈川県横浜市港北区新横浜２丁目１００番地４５",
                "神奈川県横浜市港北区新横浜2-100-45",
                "横浜市港北区新横浜二丁目百番地四十五号",
                "神奈川県横浜市港北区新横浜2丁目100-45",
            ],
            "expected": "神奈川県横浜市港北区新横浜2-100-45"
        },
        # 建物名を含む例
        {
            "inputs": [
                "東京都新宿区西新宿２ー８ー１ 東京都庁舎",
                "東京都新宿区西新宿2-8-1 都庁",
                "東京都新宿区西新宿二丁目八番一号 東京都庁",
                "新宿区西新宿2-8-1 都庁舎",
            ],
            "expected": "東京都新宿区西新宿2-8-1"
        }
    ]

    # テストケースの実行
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nテストケース {i}")
        print("=" * 50)

        # 各入力パターンで正規化を実行
        for addr in test_case["inputs"]:
            normalized = normalizer.normalize(addr)
            print(f"元の住所: {addr}")
            print(f"正規化後: {normalized}")
            print(f"期待値  : {test_case['expected']}")

            # 類似度の計算
            similarity = normalizer.compare_addresses(normalized, test_case["expected"])
            print(f"類似度  : {similarity:.2f}")
            print("-" * 50)

        # 同じテストケース内での住所同士の類似度比較
        print("\n住所同士の類似度比較:")
        for i, addr1 in enumerate(test_case["inputs"]):
            for addr2 in test_case["inputs"][i+1:]:
                similarity = normalizer.compare_addresses(addr1, addr2)
                print(f"比較: \n  {addr1}\n  {addr2}\n  類似度: {similarity:.2f}\n")


テストケース 1
元の住所: 東京都千代田区丸の内１ー１
正規化後: 東京都都千代田区丸の内1-1
期待値  : 東京都千代田区丸の内1-1
類似度  : 0.93
--------------------------------------------------
元の住所: 東京 千代田区丸の内1-1
正規化後: 東京都 千代田区丸の内1-1
期待値  : 東京都千代田区丸の内1-1
類似度  : 0.93
--------------------------------------------------
元の住所: 東京都千代田区丸の内一番地
正規化後: 東京都都千代田区丸の内1
期待値  : 東京都千代田区丸の内1-1
類似度  : 0.79
--------------------------------------------------
元の住所: 東京都千代田区丸の内1丁目1番
正規化後: 東京都都千代田区丸の内11
期待値  : 東京都千代田区丸の内1-1
類似度  : 0.86
--------------------------------------------------
元の住所: 東京都千代田区丸の内1-1-1
正規化後: 東京都都千代田区丸の内1-1-1
期待値  : 東京都千代田区丸の内1-1
類似度  : 0.82
--------------------------------------------------

住所同士の類似度比較:
比較: 
  東京都千代田区丸の内１ー１
  東京 千代田区丸の内1-1
  類似度: 0.93

比較: 
  東京都千代田区丸の内１ー１
  東京都千代田区丸の内一番地
  類似度: 0.86

比較: 
  東京都千代田区丸の内１ー１
  東京都千代田区丸の内1丁目1番
  類似度: 0.93

比較: 
  東京都千代田区丸の内１ー１
  東京都千代田区丸の内1-1-1
  類似度: 0.88

比較: 
  東京 千代田区丸の内1-1
  東京都千代田区丸の内一番地
  類似度: 0.79

比較: 
  東京 千代田区丸の内1-1
  東京都千代田区丸の内1丁目1番
  類似度: 0.86

比較: 
  東京 千代田区丸の内1-1
  東京都千代田区丸の内1-1-1
  類似